In [ ]:
%matplotlib inline

# python modules
from os.path import join
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

# my code
from download import data_exists, download
from parse import parse_labels, parse_images
from render import browse
from preprocess import process_images, process_labels

In [ ]:
#global constants
DATA_FOLDER_NAME = "data"
URL_BASE = "http://yann.lecun.com/exdb/mnist/"
FILES = {
    "train_labels": "train-labels-idx1-ubyte.gz", 
    "test_labels": "t10k-labels-idx1-ubyte.gz", 
    "train_images": "train-images-idx3-ubyte.gz", 
    "test_images": "t10k-images-idx3-ubyte.gz"
}

In [ ]:
if not data_exists(DATA_FOLDER_NAME):
    download(URL_BASE, FILES.values(), DATA_FOLDER_NAME)

In [ ]:
#make paths
train_labels_path = join(DATA_FOLDER_NAME, FILES["train_labels"])
test_labels_path = join(DATA_FOLDER_NAME, FILES["test_labels"])
train_images_path = join(DATA_FOLDER_NAME, FILES["train_images"])
test_images_path = join(DATA_FOLDER_NAME, FILES["test_images"])

#parse in as numpy arrarys
train_labels = parse_labels(train_labels_path)
test_labels = parse_labels(test_labels_path)
train_images = parse_images(train_images_path)
test_images = parse_images(test_images_path)

In [ ]:
#class labels to strings
train_labels = process_labels(train_labels)
test_labels = process_labels(test_labels)

#images normalized to interval [0,1] and flattened into vector
train_images = process_images(train_images)
test_images = process_images(test_images)

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(200,), random_state=1, verbose=True)
model.fit(train_images, train_labels)

print("Training set score: {:0.2f}".format(model.score(train_images, train_labels)))
print("Test set score: {:0.2f}".format(model.score(test_images, test_labels)))

In [ ]:
for arr in model.coefs_:
    print(arr.shape)
    
model.coefs_[0]